In [41]:
import pandas as pd
import io
import os
import requests
import mediacloud.api
import datetime as dt
import json
import hashlib
from dotenv import load_dotenv
from tqdm import tqdm
from urllib.parse import urlparse, urlunparse

In [42]:
load_dotenv()

MC_API_KEY = os.getenv("MC_API_KEY")  
RAW_ARTICLES_DIR = "raw_articles_data"   

# Create the directory if it doesn't exist
os.makedirs(RAW_ARTICLES_DIR, exist_ok=True)

In [43]:
# Initialize the Media Cloud API
try:
    search_api = mediacloud.api.SearchApi(MC_API_KEY)
    print("Media Cloud API initialized.")
except Exception as e:
    print(f"Could not initialize Media Cloud API. Check your key. Error: {e}.")
    search_api = None

Media Cloud API initialized.


In [44]:
df = pd.read_csv('data/cn-gmmp-story-pull-v1.csv')

print("Original DataFrame:")
df.head()

Original DataFrame:


,id,project_id,model_id,model_score,published_date,queued_date,processed_date,posted_date,above_threshold,model_1_score,model_2_score,source,url,normalized_url
0,38814,128,6,0.856507,2025-05-06 16:38:33.000000,2025-06-11 12:34:11.918737,2025-06-11 12:34:17.924794,2025-06-11 12:35:09.115747,True,NaN,NaN,newscatcher,https://www.yahoo.com/news/living-parents-unti...,http://yahoo.com/news/living-parents-until-pol...
1,37418,116,2,0.102506,2025-05-06 12:33:32.000000,2025-06-11 12:34:09.039428,2025-06-11 12:34:12.287421,NaN,False,NaN,NaN,newscatcher,https://lado.mx/noticia.php?id=18130041,http://lado.mx/noticia.php?id=18130041
2,37419,116,2,0.376812,2025-05-06 02:29:18.000000,2025-06-11 12:34:09.039428,2025-06-11 12:34:12.287421,NaN,False,NaN,NaN,newscatcher,https://vanguardia.com.mx/show/o-sea-que-no-ha...,http://vanguardia.com.mx/show/o-sea-que-no-hay...
3,37420,116,2,0.285902,2025-05-06 11:00:00.000000,2025-06-11 12:34:09.039428,2025-06-11 12:34:12.287421,NaN,False,NaN,NaN,newscatcher,https://heraldodemexico.com.mx/nacional/2025/5...,http://heraldodemexico.com.mx/nacional/2025/5/...
4,37421,116,2,0.219598,2025-05-06 16:34:41.000000,2025-06-11 12:34:09.039428,2025-06-11 12:34:12.287421,NaN,False,NaN,NaN,newscatcher,https://www.quien.com/espectaculos/2025/05/06/...,http://quien.com/espectaculos/2025/05/06/cuant...


In [45]:
mediacloud_df = df[df['source'] == 'media-cloud']
urls_to_fetch = mediacloud_df['url'].tolist()

print(f"Total URLs to fetch: {len(urls_to_fetch)}")

mediacloud_df.head()

# save as csv
mediacloud_df.to_csv('data/cn-gmmp-mediacloud-urls.csv', index=False)


Total URLs to fetch: 2985


In [46]:
import hashlib
import os
import json
import datetime as dt
from tqdm import tqdm
from urllib.parse import urlparse, urlunparse

def normalize_url(url: str) -> str:
    """
    Normalize URL for consistent comparison by:
    - Converting to lowercase
    - Removing trailing slashes
    - Removing common tracking parameters
    - Standardizing protocol
    """
    # Parse the URL
    parsed = urlparse(url.lower())
    
    # Remove common tracking parameters
    tracking_params = {
        'utm_source', 'utm_medium', 'utm_campaign', 'utm_term', 'utm_content',
        'fbclid', 'gclid', 'ref', 'source', 'campaign_id', '_ga', 'mc_cid'
    }
    
    # Split query parameters and filter out tracking ones
    if parsed.query:
        query_pairs = [pair for pair in parsed.query.split('&') 
                      if pair.split('=')[0] not in tracking_params]
        clean_query = '&'.join(query_pairs) if query_pairs else ''
    else:
        clean_query = ''
    
    # Remove trailing slash from path
    clean_path = parsed.path.rstrip('/')
    
    # Reconstruct URL
    normalized = urlunparse((
        parsed.scheme or 'https',  # Default to https if no scheme
        parsed.netloc,
        clean_path,
        parsed.params,
        clean_query,
        ''  # Remove fragment
    ))
    
    return normalized

def get_url_hash(url: str) -> str:
    """Generate consistent hash for normalized URL."""
    normalized_url = normalize_url(url)
    return hashlib.sha256(normalized_url.encode('utf-8')).hexdigest()

def load_processed_urls() -> dict:
    """
    Load all processed URLs from existing files into memory for fast lookup.
    Returns dict mapping normalized URLs to their file info.
    """
    processed_urls = {}
    
    if not os.path.exists(RAW_ARTICLES_DIR):
        os.makedirs(RAW_ARTICLES_DIR)
        return processed_urls
    
    print("Loading already processed URLs...")
    for filename in tqdm(os.listdir(RAW_ARTICLES_DIR), desc="Scanning files"):
        if not filename.endswith('.json'):
            continue
            
        filepath = os.path.join(RAW_ARTICLES_DIR, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                original_url = data.get('url', '')
                if original_url:
                    normalized_url = normalize_url(original_url)
                    processed_urls[normalized_url] = {
                        'filename': filename,
                        'filepath': filepath,
                        'original_url': original_url,
                        'status': data.get('status', 'unknown'),
                        'retrieved_at': data.get('retrieved_at', ''),
                        'has_text': bool(data.get('text', '').strip())
                    }
        except (json.JSONDecodeError, KeyError, Exception) as e:
            print(f"Warning: Could not read {filename}: {e}")
            continue
    
    print(f"Loaded {len(processed_urls)} already processed URLs")
    return processed_urls

def identify_failed_urls(processed_urls: dict) -> list:
    """
    Identify URLs that failed processing or have no text content.
    Returns list of URLs that could be reprocessed.
    """
    failed_urls = []
    for normalized_url, url_info in processed_urls.items():
        # Check if previous attempt failed or has no text
        if not url_info['has_text']:
            failed_urls.append(url_info['original_url'])
            continue
        
        # Check if status indicates failure
        status = url_info.get('status', '')
        if status.startswith('failed_') or status == 'not_attempted':
            failed_urls.append(url_info['original_url'])
    
    return failed_urls

In [ ]:
def get_article_text(url: str) -> dict:
    """
    Retrieves the full text of an article from a URL.
    
    Args:
        url: The URL of the article to fetch.

    Returns:
        A dictionary containing the URL, fetched text, and source of the text.
    """
    article_text = ""
    source = ""
    original_url = url
    article_title = ""
    status = "not_attempted"

    if search_api:
        # 'url:https\://www.vox.com/future-perfect/*'
        # mc_search.story_count('url:https\://www.vox.com/future-perfect/*', 
        # dt.date(2020,1,1), dt.date(2025,8,1), source_ids=[104828])
        # replace : with \:
        escaped_url = original_url.replace(':', '\:')
        # if not escaped_url.endswith('/'):
        #      escaped_url += '/'
        # escaped_url += '*' # Add wildcard to match sub-paths
        
        my_query = f'url:{escaped_url}'
        start_date = dt.date(2025, 1, 1)
        end_date = dt.date.today()
        try:
            results = search_api.story_list(my_query, start_date, end_date)
            # print(results)
            if results and len(results[0]) > 0:
                story_id = results[0][0]['id']
                article_data = search_api.story(story_id)
                article_title = article_data.get('title', '')
                article_text = article_data.get('text', '')
                source = "mediacloud"
                status = "success"
                # print(f"Successfully fetched from Media Cloud: {url}")
            else:
                status = "failed_mediacloud_no_results"
        except Exception as e:
            status = f"failed_mediacloud_exception: {type(e).__name__}: {e}"

    # If no text was retrieved, update status to reflect overall failure
    if not article_text:
        status = status if status.startswith("failed_mediacloud") else "failed_no_text_retrieved"

    return {
        "url": original_url,
        "title": article_title,
        "text": article_text,
        "source": source,
        "retrieved_at": dt.datetime.now().isoformat(),
        "status": status,
        "normalized_url": normalize_url(original_url)  
    }

def fetch_and_save_articles(urls: list, force_reprocess: bool = False):
    """
    Processes a list of URLs, fetches their text, and saves each to a unique file.
    Enhanced with better duplicate detection.
    
    Args:
        urls: List of URLs to process
        force_reprocess: If True, reprocess URLs even if they were previously successful
    """
    # Load already processed URLs
    processed_urls = load_processed_urls()
    
    failed_urls_tracker = []
    skipped_urls_tracker = []
    
    # Remove duplicates from input while preserving order
    unique_urls = []
    seen_normalized = set()
    for url in urls:
        normalized = normalize_url(url)
        if normalized not in seen_normalized:
            unique_urls.append(url)
            seen_normalized.add(normalized)
        else:
            print(f"Duplicate URL in input list (skipped): {url}")
    
    print(f"Processing {len(unique_urls)} unique URLs (removed {len(urls) - len(unique_urls)} duplicates)")
    
    for url in tqdm(unique_urls, desc="Fetching Articles", unit="article"):
        normalized_url = normalize_url(url)
        url_hash = get_url_hash(url)
        filename = url_hash + ".json"
        filepath = os.path.join(RAW_ARTICLES_DIR, filename)
        
        # Check if URL was already processed successfully
        if normalized_url in processed_urls and not force_reprocess:
            existing_info = processed_urls[normalized_url]
            # Only skip if it was successful (has text and good status)
            if existing_info['has_text'] and not existing_info['status'].startswith('failed_'):
                # print(f"Skipping already processed URL: {url}")
                skipped_urls_tracker.append(url)
                continue
        
        # If force_reprocess is True, remove existing file
        if force_reprocess and normalized_url in processed_urls:
            existing_filepath = processed_urls[normalized_url]['filepath']
            if os.path.exists(existing_filepath):
                os.remove(existing_filepath)
                print(f"Removed existing file for reprocessing: {url}")
        
        # Fetch article data
        article_data = get_article_text(url)
        
        if article_data["text"]:
            try:
                with open(filepath, 'w', encoding='utf-8') as f:
                    json.dump(article_data, f, ensure_ascii=False, indent=4)
                # print(f"Saved: {url}")
            except Exception as e:
                # Handle file write errors as a failure
                failed_urls_tracker.append({
                    "url": url, 
                    "reason": f"file_save_error: {type(e).__name__}: {e}", 
                    "status": article_data.get("status", "unknown")
                })
        else:
            # If no text was retrieved, consider it a failure for tracking
            failed_urls_tracker.append({
                "url": url, 
                "reason": article_data.get("status", "no_text_retrieved")
            })
            
            # Append to failed_urls.txt incrementally
            with open('failed_urls.txt', 'a', encoding='utf-8') as f:
                f.write(url + '\n')
    
    # Print summary
    print(f"\nProcessing Summary:")
    print(f"  Successfully processed: {len(unique_urls) - len(failed_urls_tracker) - len(skipped_urls_tracker)}")
    print(f"  Skipped (already processed): {len(skipped_urls_tracker)}")
    print(f"  Failed: {len(failed_urls_tracker)}")
    
    if failed_urls_tracker:
        print(f"\nFailed URLs:")
        for failed in failed_urls_tracker[:5]:  # Show first 5
            print(f"  {failed['url']}: {failed['reason']}")
        if len(failed_urls_tracker) > 5:
            print(f"  ... and {len(failed_urls_tracker) - 5} more")


def reprocess_failed_urls():
    """
    Identify and reprocess URLs that previously failed or have no content.
    """
    print("Identifying failed URLs...")
    processed_urls = load_processed_urls()
    failed_urls = identify_failed_urls(processed_urls)
    
    if not failed_urls:
        print("No failed URLs found to reprocess.")
        return
    
    print(f"Found {len(failed_urls)} failed URLs to reprocess.")
    
    # Show some examples
    print("Examples of failed URLs:")
    for i, url in enumerate(failed_urls[:5]):
        normalized = normalize_url(url)
        info = processed_urls[normalized]
        print(f"  {url}")
        print(f"    Status: {info['status']}, Has text: {info['has_text']}")
    
    if len(failed_urls) > 5:
        print(f"  ... and {len(failed_urls) - 5} more")
    
    # Ask for confirmation
    response = input(f"\nReprocess {len(failed_urls)} failed URLs? (y/N): ")
    if response.lower() != 'y':
        print("Reprocessing cancelled.")
        return
    
    # Reprocess with force flag
    fetch_and_save_articles(failed_urls, force_reprocess=True)

# Example usage:

# Normal processing (skips already successful URLs)
# fetch_and_save_articles(['https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/'])

# Force reprocess specific URLs
# fetch_and_save_articles(['https://example.com/article'], force_reprocess=True)

# Identify and reprocess all failed URLs interactively
# reprocess_failed_urls()

# Get list of failed URLs programmatically
# processed_urls = load_processed_urls()
# failed_urls = identify_failed_urls(processed_urls)
# print(f"Found {len(failed_urls)} failed URLs")

Loading already processed URLs...


Scanning files: 100%|██████████| 1665/1665 [00:00<00:00, 13887.02it/s]

Loaded 1665 already processed URLs
Found 0 failed URLs


In [48]:
print("\n--- Starting Article Fetching ---")
fetch_and_save_articles(urls_to_fetch)
print("\n--- Article Fetching Complete ---")


--- Starting Article Fetching ---
Loading already processed URLs...


Scanning files: 100%|██████████| 1664/1664 [00:00<00:00, 4926.84it/s]


Loaded 1664 already processed URLs
Processing 2985 unique URLs (removed 0 duplicates)


Fetching Articles:   0%|          | 5/2985 [00:05<55:10,  1.11s/article]


KeyboardInterrupt: 

In [50]:
with open('failed_urls.txt', 'r', encoding='utf-8') as f:
    failed_urls = [line.strip() for line in f if line.strip()]

fetch_and_save_articles(failed_urls, force_reprocess=True)

Loading already processed URLs...


Scanning files: 100%|██████████| 1665/1665 [00:00<00:00, 11279.34it/s]


Loaded 1665 already processed URLs
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/
Duplicate URL in input list (skipped): https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary
Duplicate URL in input list (skipped): https://chicago.suntimes.co

Fetching Articles:   0%|          | 0/1321 [00:00<?, ?article/s]

Removed existing file for reprocessing: https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/


Fetching Articles: 100%|██████████| 1321/1321 [20:59<00:00,  1.05article/s]


Processing Summary:
  Successfully processed: 1309
  Skipped (already processed): 0
  Failed: 12

Failed URLs:
  https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary/: failed_mediacloud_no_results
  https://radaronline.com/p/sydney-sweeney-engaged-patrick-schwarzenegger-intimate-moments: failed_mediacloud_no_results
  https://www.nbcchicago.com/news/local/surreal-family-friends-remember-suburban-teen-killed-in-crash-after-senior-prom/3738924: failed_mediacloud_no_results
  https://radaronline.com/p/bill-murray-blasted-hollywood-grouchy-poor-me-behavior: failed_mediacloud_no_results
  https://www.thenation.com/article/society/joshua-clover-obituary-uc-davis: failed_mediacloud_no_results
  ... and 7 more


In [51]:
import pandas as pd
import json
import os
from pathlib import Path
import hashlib
from tqdm import tqdm

def normalize_url(url):
    """Normalize URL for consistent matching (should match your original function)"""
    if not url:
        return url
    
    # Remove trailing slash
    url = url.rstrip('/')
    
    # Convert to lowercase
    url = url.lower()
    
    # Remove common tracking parameters
    tracking_params = ['utm_source', 'utm_medium', 'utm_campaign', 'utm_content', 'utm_term', 
                      'fbclid', 'gclid', 'ref', 'source']
    
    if '?' in url:
        base_url, params = url.split('?', 1)
        param_pairs = params.split('&')
        filtered_params = []
        
        for param in param_pairs:
            if '=' in param:
                key = param.split('=')[0]
                if key not in tracking_params:
                    filtered_params.append(param)
        
        if filtered_params:
            url = base_url + '?' + '&'.join(filtered_params)
        else:
            url = base_url
    
    return url

def get_url_hash(url):
    """Generate hash for URL (should match your original function)"""
    normalized = normalize_url(url)
    return hashlib.md5(normalized.encode('utf-8')).hexdigest()

def load_article_data(json_dir):
    """
    Load all article JSON files and create a mapping from normalized URL to article data.
    
    Args:
        json_dir: Directory containing the JSON files
        
    Returns:
        Dictionary mapping normalized URLs to article data
    """
    articles = {}
    json_files = list(Path(json_dir).glob("*.json"))
    
    print(f"Loading {len(json_files)} article files...")
    
    for json_file in tqdm(json_files, desc="Loading articles"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                article_data = json.load(f)
            
            # Use normalized_url if available, otherwise normalize the url
            if 'normalized_url' in article_data:
                normalized_url = article_data['normalized_url']
            else:
                normalized_url = normalize_url(article_data.get('url', ''))
            
            articles[normalized_url] = article_data
            
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    return articles

def merge_csv_with_articles(csv_path, json_dir, output_path=None):
    """
    Merge CSV data with article text from JSON files.
    
    Args:
        csv_path: Path to the original CSV file
        json_dir: Directory containing article JSON files
        output_path: Path for output CSV (optional, will auto-generate if not provided)
    
    Returns:
        DataFrame with merged data
    """
    # Load the original CSV
    print("Loading original CSV...")
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} rows from CSV")
    
    # Load article data
    articles = load_article_data(json_dir)
    print(f"Loaded {len(articles)} article files")
    
    # Initialize new columns
    df['article_title'] = ''
    df['article_text'] = ''
    df['article_source'] = ''
    df['article_status'] = ''
    df['article_retrieved_at'] = ''
    df['has_article_text'] = False
    
    # Track matching statistics
    matched_count = 0
    unmatched_urls = []
    
    # Merge data
    print("Merging article data with CSV...")
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Merging data"):
        # Try to find article data using normalized_url first, then url
        article_data = None
        
        if pd.notna(row.get('normalized_url')):
            article_data = articles.get(row['normalized_url'])
        
        if not article_data and pd.notna(row.get('url')):
            # Try with the original URL normalized
            normalized_original = normalize_url(row['url'])
            article_data = articles.get(normalized_original)
        
        if article_data:
            df.at[idx, 'article_title'] = article_data.get('title', '')
            df.at[idx, 'article_text'] = article_data.get('text', '')
            df.at[idx, 'article_source'] = article_data.get('source', '')
            df.at[idx, 'article_status'] = article_data.get('status', '')
            df.at[idx, 'article_retrieved_at'] = article_data.get('retrieved_at', '')
            df.at[idx, 'has_article_text'] = bool(article_data.get('text', '').strip())
            matched_count += 1
        else:
            df.at[idx, 'article_status'] = 'not_found_in_json'
            unmatched_urls.append(row.get('url', 'No URL'))
    
    # Print statistics
    print(f"\nMerging Statistics:")
    print(f"  Total rows: {len(df)}")
    print(f"  Matched with article data: {matched_count}")
    print(f"  Unmatched: {len(df) - matched_count}")
    print(f"  Rows with article text: {df['has_article_text'].sum()}")
    
    if unmatched_urls:
        print(f"\nFirst 5 unmatched URLs:")
        for url in unmatched_urls[:5]:
            print(f"  {url}")
        if len(unmatched_urls) > 5:
            print(f"  ... and {len(unmatched_urls) - 5} more")
    
    # Save to CSV if output path provided
    if output_path:
        print(f"\nSaving merged data to {output_path}...")
        df.to_csv(output_path, index=False)
        print("Saved successfully!")
    
    return df

def analyze_merge_results(df):
    """
    Analyze the results of the merge operation.
    """
    print("\n" + "="*50)
    print("MERGE ANALYSIS")
    print("="*50)
    
    total_rows = len(df)
    has_text = df['has_article_text'].sum()
    no_text = total_rows - has_text
    
    print(f"Total articles: {total_rows}")
    print(f"With text: {has_text} ({has_text/total_rows*100:.1f}%)")
    print(f"Without text: {no_text} ({no_text/total_rows*100:.1f}%)")
    
    # Status breakdown
    print(f"\nStatus breakdown:")
    status_counts = df['article_status'].value_counts()
    for status, count in status_counts.items():
        print(f"  {status}: {count} ({count/total_rows*100:.1f}%)")
    
    # Text length statistics for articles with text
    text_lengths = df[df['has_article_text']]['article_text'].str.len()
    if len(text_lengths) > 0:
        print(f"\nArticle text length statistics (for articles with text):")
        print(f"  Mean: {text_lengths.mean():.0f} characters")
        print(f"  Median: {text_lengths.median():.0f} characters")
        print(f"  Min: {text_lengths.min()} characters")
        print(f"  Max: {text_lengths.max()} characters")


# Update these paths according to your setup
CSV_PATH = "data/cn-gmmp-mediacloud-urls.csv"  # Path to your original CSV
JSON_DIR = "raw_articles_data"  # Directory containing your JSON files
OUTPUT_PATH = "merged_articles.csv"  # Output file path

# Perform the merge
merged_df = merge_csv_with_articles(CSV_PATH, JSON_DIR, OUTPUT_PATH)

# Analyze results
analyze_merge_results(merged_df)

Loading original CSV...
Loaded 2985 rows from CSV
Loading 2973 article files...


Loading articles: 100%|██████████| 2973/2973 [00:00<00:00, 5343.90it/s]


Loaded 2973 article files
Merging article data with CSV...


Merging data: 100%|██████████| 2985/2985 [00:00<00:00, 6696.10it/s]



Merging Statistics:
  Total rows: 2985
  Matched with article data: 2865
  Unmatched: 120
  Rows with article text: 2865

First 5 unmatched URLs:
  https://chicago.suntimes.com/obituaries/2025/05/06/michael-miner-dead-media-columnist-chicago-reader-obituary
  https://hotspotsmagazine.com/2025/05/06/queer-music-trailblazer-jill-sobule-dies-at-66-in-house-fire/?utm_source=rss&utm_medium=rss&utm_campaign=queer-music-trailblazer-jill-sobule-dies-at-66-in-house-fire
  https://www.seattletimes.com/entertainment/singer-songwriter-lizzy-mcalpine-makes-her-broadway-debut-coming-at-a-perfect-time/?utm_source=RSS&utm_medium=Referral&utm_campaign=RSS_all
  https://www.seattletimes.com/nation-world/nation/a-philadelphia-woman-is-the-8th-person-to-die-from-the-january-crash-of-a-medical-plane/?utm_source=RSS&utm_medium=Referral&utm_campaign=RSS_all
  https://www.seattletimes.com/entertainment/movies/siff-2025-seattles-premier-film-festival-returns-in-challenging-times/?utm_source=RSS&utm_medium=Ref